# Рекомендательные системы. Введение

## Практическая работа

**Задание 1)** Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)

Поскольку "наши знания" в данном случае нулевые, пара идей взята из Википедии https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)

Для понимания нужно "перевести" суть метрик с языка поисковых систем на язык интернет-магазина, а именно:
- "retrieved documents" (поисковая выдача) становятся "рекомендованными товарами"
- "relevant documents" - это в данном случае "купленные товары" (или релевантные в другом смысле, например, положенные в корзину, просмотренные в течение определенного времени и т.п.)

В статье, помимо Precision и Recall, упоминаются другие метрики, образованные пересечением "рекомендованных" и "релевантных" множеств, не учитывающие порядок (ранг) рекомендаций:
- Fall-out - доля рекомендованных, но не купленных, товаров по отношению ко всем не купленным (своеобразный аналог False Positive Rate)
- F-score - аналогично бинарной классификации

Многие статьи в Интернете делают акцент не столько на качестве предсказания интересов клиента, сколько на способах представить рекомендательную систему интеллектуальным собеседником, способным привнести в жизнь новое и неожиданное. 

Отсюда появление специальных категорий метрик:
- Diversity (разносторонность) - оценивает степень непохожести рекомендованных элементов для пользователя. Выбор технической реализации метрики зависит от того, какие признаки описывают элементы. Могут использоваться различные коэффициенты сходства, метрики расстояний.
- Coverage (широта покрытия) - оценивает способность рекомендательной системы отражать широту ассортимента в рекомендациях, в частности, предлагать элементы без рейтинга и с короткой историей пользовательских транзакций.
- Serendipity (способность удачно находить то, что не искал). Оценивается свойство системы предложить персонализированную релевантную рекомендацию не из набора универсальных рекомендаций, которые пользователь ожидает увидеть в списке.
- Novelty (новизна) - способность предложить пользователю то, что он до этого не видел.

**Задание 2)** Доделать все функции, где стоит комментарий "сделать дома"

**Примечание**: Переписал код функций, скорее для лучшего понимания, нежели для оптимизации.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Метрики будем проверять на следующих данных
recommended_list_users = [
    [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],
    [1134, 533, 14, 4, 1, 1543, 15, 99, 27, 3345],
    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11],
    [4, 22, 21, 1, 234, 232432, 234234, 666]
]

bought_list_users = [
    [521, 32, 143, 991],
    [143, 156, 991, 43, 1],
    [1, 2],
    [1, 221, 3, 4, 5, 6, 7, 8, 9]
]

### Hit Rate

In [3]:
def hit_rate(recommended_list, bought_list):
    _b_and_r = np.intersect1d(bought_list, recommended_list)
    return np.sign(_b_and_r.size)

def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)

In [4]:
assert hit_rate_at_k(recommended_list_users[0], bought_list_users[0], k=3) == 1
assert hit_rate_at_k(recommended_list_users[1], bought_list_users[1], k=3) == 0

### Precision
> Доля купленных товаров среди рекомендованных

In [5]:
def precision(recommended_list, bought_list):
    _b_and_r = np.intersect1d(bought_list, recommended_list)
    return _b_and_r.size / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [6]:
assert precision_at_k(recommended_list_users[0], bought_list_users[0], k=30) == 0.2
assert precision_at_k(recommended_list_users[0], bought_list_users[0], k=3) == 1/3

In [7]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    top_rec = recommended_list[:k]
    top_prices = prices_recommended[:k]
    _r_in_b = np.isin(top_rec, bought_list)
    return np.dot(_r_in_b, top_prices) / np.sum(top_prices)

In [8]:
prices_rec = [400, 60, 40, 40, 90]
money_precision_at_k(recommended_list_users[0], bought_list_users[0], prices_rec, k=3)

0.8

### Recall
> Доля рекомендованных товаров среди купленных

In [9]:
def recall(recommended_list, bought_list):
    _b_and_r = np.intersect1d(bought_list, recommended_list)
    return _b_and_r.size / len(bought_list)

def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)

In [10]:
assert recall_at_k(recommended_list_users[0], bought_list_users[0], k=30) == 0.5
assert recall_at_k(recommended_list_users[0], bought_list_users[0], k=3) == 0.25

In [11]:
def money_recall_at_k(recommended_list, bought_list, prices_bought, k=5):
    top_rec = recommended_list[:k]
    _b_in_r = np.isin(bought_list, top_rec)
    return np.dot(_b_in_r, prices_bought) / np.sum(prices_bought)

In [12]:
prices_bought = [20, 60, 40, 40]
money_recall_at_k(recommended_list_users[0], bought_list_users[0], prices_bought, k=3)

0.25

### AP@k

In [13]:
def ap_k(recommended_list, bought_list, k=5):
    top_rec = recommended_list[:k]
    bought_set = set(bought_list)
    
    n_relevant = 0
    sum_precision = 0.0
    
    for n_rec, rec in enumerate(top_rec, start=1):
        if rec in bought_set:
            n_relevant += 1
            sum_precision += n_relevant / n_rec
    
    return sum_precision / n_relevant if sum_precision else 0.0

In [14]:
ap_k(recommended_list_users[-1], bought_list_users[-1])

0.75

### MAP@k

In [15]:
def map_k(recommended_list_users, bought_list_users, k=5):
    _gen_ap_k = (
        ap_k(recommended_list, bought_list, k) 
        for recommended_list, bought_list
        in zip(recommended_list_users, bought_list_users)
    )
    return np.mean(np.fromiter(_gen_ap_k, float))

In [16]:
map_k(recommended_list_users, bought_list_users, k=5)

0.425

### Normalized discounted cumulative gain ( NDCG@k)

In [17]:
# Реализовал дисконт, как в Википедии (попроще), а не как в методичке

In [18]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    top_rec = recommended_list[:k]
    discount = np.log2(np.arange(k) + 2)
    
    _r_in_b = np.isin(top_rec, bought_list)
    return np.sum(_r_in_b / discount) / np.sum(1 / discount)

In [19]:
ndcg_at_k(recommended_list_users[0], bought_list_users[0], k=5)

0.48522855511632257

### Mean Reciprocal Rank ( MRR@k )
> Reciprocal Rank - обратная величина к рангу первого релевантного предсказания

In [20]:
def reciprocal_rank(recommended_list, bought_list, k):
    top_rec = recommended_list[:k]
    bought_set = set(bought_list)
    
    rank = 0
    for rec_rank, rec in enumerate(top_rec, start=1):
        if rec in bought_set:
            rank = rec_rank
            break
    return 1 / rank if rank else 0.0

In [21]:
reciprocal_rank(recommended_list_users[1], bought_list_users[1], k=50)

0.2

In [22]:
def mean_reciprocal_rank(recommended_list_users, bought_list_users, k=5):
    _gen_reciprocal_rank = (
        reciprocal_rank(recommended_list, bought_list, k) 
        for recommended_list, bought_list
        in zip(recommended_list_users, bought_list_users)
    )
    return np.mean(np.fromiter(_gen_reciprocal_rank, float))

In [23]:
mean_reciprocal_rank(recommended_list_users, bought_list_users, k=5)

0.55